In [235]:
#This script extracts the discrete metrics from the resampled kinetic_kinematic data

#import libraries
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

#import Simpson's integration function
from scipy.integrate import simps

In [239]:
#set the working directory
os.chdir('/Users/rossbrancati/Desktop/PFP_Project/kinetic_kinematic/resampled_data')
data_directory = os.path.join(os.getcwd(), 'injured_resampled')
directory = sorted(os.listdir(data_directory))
#directory.remove('.DS_Store')

#load the excel file for storing the discrete metrics
#discrete_data_sheet = pd.read_excel(os.path.join(os.getcwd(),'discrete_data','discrete_data_master.xlsx'))
#print(discrete_data_sheet)

#create a dataframe for storing discrete metrics
discrete_data_frame = pd.DataFrame(columns=['peak_hip_rot_stance', 'peak_hip_add_stance', 'peak_knee_int_rot_stance', 'peak_knee_flex_stance',
                        'knee_ext_rot_at_knee_ext_mom', 'peak_knee_ext_moment_stance', 'peak_knee_add_mom_LR', 
                        'knee_abduction_impulse_stance', 'peak_knee_ext_rot_mom_LR', 'peak_knee_ext_mom_LR', 
                        'knee_ext_mom_toe_off', 'peak_knee_adduction_angle_stance', 'rearfoot_eversion_heel_strike', 
                        'rearfoot_eversion_ROM_ten_percent', 'rearfoot_eversion_ROM_stance'])

In [240]:
#loop through files in directory

for file_idx,file_name in enumerate(directory):
    
    #set variable to file path 
    file_path = os.path.join(data_directory,file_name)
    
    #load the file
    raw_file = pd.read_csv(file_path,sep='\t')
    _,num_cols = raw_file.shape
    
    #count the number of strides
    stride_count = int(num_cols/18)
    
    #loop through each stride
    #***This may have to be switched to stride_count-1 because indexing starts at 0***
    for i in range(stride_count):
        
        #get index for toe off
        toe_off = raw_file.iloc[:,3*stride_count+i][20:60].idxmin()
        
        #peak hip rotation angle during stance
        peak_hip_rot_stance = max(raw_file.iloc[:toe_off,2*stride_count+i])
        
        #peak hip adduction angle during stance
        #peak hip adduction angle should be positive according to Dierks 2010
        peak_hip_add_stance = max(raw_file.iloc[:toe_off,stride_count+i])
        
        #peak_knee_internal_rotation_stance
        #knee internal rotation should be positive according to Dierks 2010, hence the min function and multiplication by -1
        peak_knee_int_rot_stance = min(raw_file.iloc[:toe_off,5*stride_count+i])*-1
        
        #peak knee flexion angle stance
        peak_knee_flex_stance = max(raw_file.iloc[:toe_off,3*stride_count+i])
        
        #knee external rotation angle at peak knee extension moment
        peak_knee_ext_mom_idx = raw_file.iloc[:,3*stride_count+i][0:toe_off].idxmax()
        knee_ext_rot_at_knee_ext_mom = raw_file.iloc[peak_knee_ext_mom_idx,5*stride_count+i]
        
        #peak knee extension moment stance
        #peak knee extension moment stance should be positive
        #SHOULD THIS BE PEAK KNEE FLEXION MOMENT BECAUSE WE ARE ALREADY GOING TO CAPTURE THE PEAK WITH
        #THE PEAK KNEE EXTENSION MOMENT DURING LOADING PHASE
        peak_knee_ext_moment_stance = max(raw_file.iloc[:toe_off,12*stride_count+i])
        
        #peak knee adduction moment during loading response
        #loading response occurs in first 0-12% of stance, so those will be the indices
        #knee abduction moment should be positive according to Paoloni 2010, so taking min and multiplying by -1
        peak_knee_add_mom_LR = min(raw_file.iloc[0:12,13*stride_count+i])*-1
        
        #knee abduction moment impulse during stance
        #impulse is defined as the area under the curve
        #multiplied by -1 because the signal is all negative, so AOC is a negative value
        #in Stefanyshyn 2006, this metric is positive
        knee_abduction_impulse_stance_signal = raw_file.iloc[:toe_off,13*stride_count+i]
        knee_abduction_impulse_stance = simps(knee_abduction_impulse_stance_signal, dx=5)*-1
        
        #peak knee external rotation moment loading response 
        peak_knee_ext_rot_mom_LR = max(raw_file.iloc[0:12,14*stride_count+i])
        
        #peak knee extension moment during loading response
        #I am not seeing the slight dip to capture the local min of peak extension moment
        #taking hte minimum and multiplying by -1 because values in Paoloni 2010 were all positive
        peak_knee_ext_mom_LR = min(raw_file.iloc[0:12,12*stride_count+i])
        
        #knee extension moment at toe off
        knee_ext_mom_toe_off = raw_file.iloc[toe_off,12*stride_count+i]
        
        #peak knee adduction angle stance
        peak_knee_adduction_angle_stance = min(raw_file.iloc[0:toe_off,4*stride_count+i])
        
        #rearfoot eversion at heel strike
        rearfoot_eversion_heel_strike = raw_file.iloc[0,7*stride_count+i]
        
        #rearfoot eversion ROM in first 10% of stance
        rearfoot_eversion_ROM_ten_percent = (raw_file.iloc[10,7*stride_count+i] - raw_file.iloc[0,7*stride_count+i])*-1
        
        #rearfoot eversion ROM stance
        rearfoot_eversion_ROM_stance = (raw_file.iloc[toe_off,7*stride_count+i] - raw_file.iloc[0,7*stride_count+i])*-1

        #create a vector of all the metrics of interest
        metric_vector = pd.DataFrame([[peak_hip_rot_stance, peak_hip_add_stance, peak_knee_int_rot_stance, peak_knee_flex_stance,
                        knee_ext_rot_at_knee_ext_mom, peak_knee_ext_moment_stance, peak_knee_add_mom_LR, 
                        knee_abduction_impulse_stance, peak_knee_ext_rot_mom_LR, peak_knee_ext_mom_LR, 
                        knee_ext_mom_toe_off, peak_knee_adduction_angle_stance, rearfoot_eversion_heel_strike, 
                        rearfoot_eversion_ROM_ten_percent, rearfoot_eversion_ROM_stance]], columns=['peak_hip_rot_stance', 'peak_hip_add_stance', 'peak_knee_int_rot_stance', 'peak_knee_flex_stance',
                        'knee_ext_rot_at_knee_ext_mom', 'peak_knee_ext_moment_stance', 'peak_knee_add_mom_LR', 
                        'knee_abduction_impulse_stance', 'peak_knee_ext_rot_mom_LR', 'peak_knee_ext_mom_LR', 
                        'knee_ext_mom_toe_off', 'peak_knee_adduction_angle_stance', 'rearfoot_eversion_heel_strike', 
                        'rearfoot_eversion_ROM_ten_percent', 'rearfoot_eversion_ROM_stance'])
        
        #append the discrete data sheet
        discrete_data_frame = discrete_data_frame.append(metric_vector,ignore_index=True)
        
    print(stride_count)

0
12
12
12
13
14
15
13
13
13
13
13
13
13
13
13
13
14
13
13
12
12
13
12
13
13
13
13
13
13
13
13
12
13
13
12
12


In [241]:
#write the discrete_data_frame to a .csv file
discrete_data_frame.to_csv('/Users/rossbrancati/Desktop/PFP_Project/kinetic_kinematic/discrete_data/discrete_data_injured.csv')

In [242]:
print(file_name)

S56_kinetic_kinematic_021.txt
